In [72]:
import numpy as np
import pandas as pd

In [143]:
def monte_carlo_pricing_method_american_put_option(S_init, K, rf, sigma, T, n, num_of_ite, option_type):
    dt = T / n # number of time steps
    option_prices = np.zeros(num_of_ite)
    S_t = np.full((num_of_ite, n), S_init)   
    df = np.exp(-rf * dt) 
    
    # underlying stock price evolution
    for i in range(num_of_ite):
        for l in range(1, S_t.shape[1]):
            epsilon = np.random.randn()
            S_t[ i, l] = S_t[i, l-1] * np.exp((rf - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * epsilon)
    
    # create immediate payoff matrix if we exercise the put option at any time step for any iteration
    payoff_mat = np.clip(K - S_t, a_min=0, a_max=None) # change this for call option
    
    # create cash flow matrix starting from the end and then we will proceed towards inner time steps using algorithm
    # we are trying to compute expected cash flow if we dont exercise the option
    cf = payoff_mat.copy()
    cf[:] = 0
    cf[:,n-1] = payoff_mat[:, n-1]
    
    cf = pd.DataFrame(cf)
    S_t = pd.DataFrame(S_t)
    payoff_mat = pd.DataFrame(payoff_mat)
    
    for s in range(n-2,0,-1):
        y_array = cf.iloc[:,s+1]*df
        x_array = S_t.iloc[:, s]
        table = pd.DataFrame({"Y":y_array, "X":x_array})
        ind_possible_exercise = S_t[S_t.iloc[:, s] < K].index # find index that can be exercised
        table_t_inmoney=table.loc[ind_possible_exercise] # only those where possible exercise can be done
        rg_t = np.polyfit(table_t_inmoney["X"], table_t_inmoney["Y"], 2) # fit polynomial to predict future expected cashflow based on current stock price
        exp_future_payoff = np.polyval(rg_t, S_t.loc[ind_possible_exercise].iloc[:,s]) # predict continuation expected payoff
        cf.loc[ind_possible_exercise,s] = np.where(payoff_mat.loc[ind_possible_exercise,s] > exp_future_payoff, payoff_mat.loc[ind_possible_exercise,s], 0)   # fill cashflow based on exercise vs expected payoff
    # below set of loop is if we exercise early, then we have to make all future cashflow for that iteration zero
        for tt in range(s, n-1):
            cf.loc[ind_possible_exercise,tt+1] = np.where(cf.loc[ind_possible_exercise,s] > 0, 0, cf.loc[ind_possible_exercise,tt+1])
    
    Sum_DCF = 0
    
    # discount all future cash flows to present value by multiplying df raise to whatever time steps we exercise option
    for t in range(n-1,0,-1):
        Sum_DCF = sum(cf.loc[:,t])*np.exp(-dt*rf*t) + Sum_DCF

    Option_Value = Sum_DCF/num_of_ite

    return cf, S_t, Opt

In [157]:
S_init = 100
K = 100
rf = 0.05
sigma = 0.2
T = 0.25
n = int(T*252)
num_of_ite = 10000

In [158]:
cf, S = monte_carlo_pricing_method_american_put_option(S_init, K, rf, sigma, T, n, num_of_ite, "P")

In [153]:
S.loc[:,0].sum()

1000000

In [154]:
S

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,100,101,97,99,97,97,97,95,94,93,...,70,70,69,69,68,68,67,67,67,67
1,100,98,98,98,97,97,95,95,93,93,...,66,64,64,64,62,61,60,59,58,57
2,100,100,100,99,100,100,100,100,100,99,...,75,75,74,75,75,72,70,69,69,68
3,100,98,98,98,98,94,92,92,90,89,...,58,58,57,58,58,58,58,56,55,54
4,100,99,98,97,96,95,94,92,93,93,...,69,67,65,65,65,64,63,63,62,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,100,99,100,100,103,102,101,103,104,103,...,86,87,87,88,89,88,89,88,87,89
9996,100,100,99,98,98,95,94,94,93,90,...,61,60,60,60,59,59,60,61,61,60
9997,100,99,97,98,96,95,93,94,92,93,...,80,79,79,79,79,79,79,78,77,79
9998,100,99,101,100,98,98,97,98,95,95,...,81,80,77,76,76,76,75,75,73,73
